<img src="http://oproject.org/tiki-download_file.php?fileId=8&display&x=450&y=128" width="200" height="200">
<img src="http://gfif.udea.edu.co/root/tmva/img/tmva_logo.gif" width="200" height="200">

# Autoencoder Variable Transformation
<hr style="border-top-width: 4px; border-top-color: #34609b;"> 

### Initialize DataLoader and Factory 

In [1]:
TMVA::Tools::Instance();
TFile *inputFile = TFile::Open( "../datasets/mydataset.root"); 
TFile* outputFile = TFile::Open( "mydataset_output.root", "RECREATE" );
TMVA::Factory *factory = new TMVA::Factory("TMVARegression", outputFile, 
                                           "!V:!Silent:Color:DrawProgressBar:AnalysisType=Regression" );
TMVA::DataLoader *loader=new TMVA::DataLoader("mydataset");

--- Factory                  : You are running ROOT Version: 6.07/07, Apr 1, 2016
--- Factory                  : 
--- Factory                  : _/_/_/_/_/ _|      _|  _|      _|    _|_|   
--- Factory                  :    _/      _|_|  _|_|  _|      _|  _|    _| 
--- Factory                  :   _/       _|  _|  _|  _|      _|  _|_|_|_| 
--- Factory                  :  _/        _|      _|    _|  _|    _|    _| 
--- Factory                  : _/         _|      _|      _|      _|    _| 
--- Factory                  : 
--- Factory                  : ___________TMVA Version 4.2.1, Feb 5, 2015
--- Factory                  : 


### Setup DataLoader

In [ ]:
loader->AddVariable( "var1", 'F' );
loader->AddVariable( "var2", "Variable 2", "", 'F' );
loader->AddVariable( "var3", "Variable 3", "units", 'F' );
loader->AddVariable( "var4", "Variable 4", "units", 'F' );
TString fname = "../datasets/tmva_example_multiple_background.root";
input = TFile::Open( fname );
TTree *signal      = (TTree*)input->Get("TreeS");
TTree *background0 = (TTree*)input->Get("TreeB0");
TTree *background1 = (TTree*)input->Get("TreeB1");
TTree *background2 = (TTree*)input->Get("TreeB2");
loader->AddTree(signal,"Signal");
loader->AddTree(background0,"bg0");
loader->AddTree(background1,"bg1");
loader->AddTree(background2,"bg2");
loader->PrepareTrainingAndTestTree( "", "SplitMode=Random:NormMode=NumEvents:!V" );

input_line_70:8:8: error: redefinition of 'signal' as different kind of symbol
TTree *signal      = (TTree*)input->Get("TreeS");
       ^
/usr/include/sys/signal.h:390:8: note: previous definition is here
void    (*signal(int, void (*)(int)))(int);
          ^


### Get DataSet Info

In [ ]:
loader->CalcNorm()

### Setup Deep Neural Network

In [ ]:
TString layoutString ("Layout=TANH|3,LINEAR");
TString training0 ("LearningRate=1e-1,Momentum=0.0,Repetitions=1,ConvergenceSteps=300,BatchSize=20,TestRepetitions=15,WeightDecay=0.001,Regularization=NONE,DropConfig=0.0+0.5+0.5+0.5,DropRepetitions=1,Multithreading=True");
TString training1 ("LearningRate=1e-2,Momentum=0.5,Repetitions=1,ConvergenceSteps=300,BatchSize=30,TestRepetitions=7,WeightDecay=0.001,Regularization=L2,Multithreading=True,DropConfig=0.0+0.1+0.1+0.1,DropRepetitions=1");
// TString training2 ("LearningRate=1e-2,Momentum=0.3,Repetitions=1,ConvergenceSteps=300,BatchSize=40,TestRepetitions=7,WeightDecay=0.0001,Regularization=L2,Multithreading=True");
// TString training3 ("LearningRate=1e-3,Momentum=0.1,Repetitions=1,ConvergenceSteps=200,BatchSize=70,TestRepetitions=7,WeightDecay=0.0001,Regularization=NONE,Multithreading=True");

// TString training0 ("LearningRate=1e-5,Momentum=0.5,Repetitions=1,ConvergenceSteps=500,BatchSize=50,TestRepetitions=7,WeightDecay=0.01,Regularization=NONE,DropConfig=0.5+0.5+0.5+0.5,DropRepetitions=2,Multithreading=True");
// TString training1 ("LearningRate=1e-5,Momentum=0.9,Repetitions=1,ConvergenceSteps=170,BatchSize=30,TestRepetitions=7,WeightDecay=0.01,Regularization=L2,DropConfig=0.1+0.1+0.1,DropRepetitions=1,Multithreading=True");
// TString training2 ("LearningRate=1e-5,Momentum=0.3,Repetitions=1,ConvergenceSteps=150,BatchSize=40,TestRepetitions=7,WeightDecay=0.01,Regularization=NONE,Multithreading=True");
// TString training3 ("LearningRate=1e-6,Momentum=0.1,Repetitions=1,ConvergenceSteps=500,BatchSize=100,TestRepetitions=7,WeightDecay=0.0001,Regularization=NONE,Multithreading=True");

TString trainingStrategyString ("TrainingStrategy=");
// trainingStrategyString += training0 + "|" + training1 + "|" + training2 + "|" + training3;
trainingStrategyString += training0 + "|" + training1;
TString nnOptions ("AE(indexLayer=1;pretraining=false;!H:V:ErrorStrategy=SUMOFSQUARES:VarTransform=G:WeightInitialization=XAVIERUNIFORM");
nnOptions.Append (":");
nnOptions.Append (layoutString);
nnOptions.Append (":");
nnOptions.Append (trainingStrategyString);
nnOptions.Append (")");

In [ ]:
cout << nnOptions.Data() << endl;

### Apply Autoencoder Transform

In [ ]:
TMVA::DataLoader* newloader = loader->VarTransform(nnOptions);